In [1]:
from __future__ import annotations
from oop_functions.classifier_oop import *
from oop_functions.util_functions import *
from oop_functions.merge_dataset_functions import *
from oop_functions.analytics_oop import *

from xgboost import XGBClassifier

In [2]:
processed_data = pd.read_csv('./processed_dataset/recent_propagated_dataset.csv', index_col=0)

In [ ]:
year = 1
label = f'cancer_in_next_{year}_years'
source_df = processed_data
source_df = resample_class(source_df, label, 0, 30000)

In [ ]:
classifier = XGBClassifier(eval_metric= 'error', learning_rate= 0.1)
experiment_data_helper = ExperimentDataHelper1(source_df, label, ['cancer_'])
data_util_lambdas = experiment_data_helper.train_test_split_util.split_kfold(5)
data_util = data_util_lambdas[0]()

In [ ]:
analytics_util = AnalyticsUtil(classifier, data_util).fit()

In [ ]:
analytics_util.get_report_generation_util().display_graph()

In [ ]:
# Get cv split
# run cv classifiers
# aggregate cv results
# visualize cv results
# save cv results
class CvAnalyticsUtil:
    def __init__(self, analytics_utils: List[AnalyticsUtil]) -> None:
        self.analytics_utils = analytics_utils

    def get_cv_report(self):
        cv_scores = []
        for k, analytics_util in enumerate(self.analytics_utils):
            report = analytics_util.get_report_generation_util().generate_report()
            cv_scores.append(report)
        cv_scores = pd.concat(cv_scores)
        cv_scores = cv_scores.reset_index()
        cv_scores = cv_scores.drop('index', axis=1)
        measures_df = cv_scores.describe().T[['mean', 'std', 'min', 'max']]
        print('\n\nCross-Validation measures:')
        print_df(measures_df)
        return cv_scores, measures_df

    def calc_confusion_matrix(self):
        pass

    def roc_with_interval(self):
        pass

    def precision_recall_with_intervals(self):
        pass

class FeatureImportanceCvAnalyticsUtil(CvAnalyticsUtil):
    def get_cv_feature_selection(self):
        df_feature_importance_tree = None
        for k, analytics_util in enumerate(self.analytics_utils):
            fn=analytics_util.data_util.get_feature_names()
            top_feature_stats, feature_importances = analytics_util.feature_selection()
            feature_importances = feature_importances[feature_importances['importance'] > 0]
            if df_feature_importance_tree is not None:
                df_feature_importance_tree = df_feature_importance_tree.merge(feature_importances, on='column_name', how='outer', suffixes=[f'_tiral_{k}', f'_tiral_{k+1}'])
            else:
                df_feature_importance_tree = feature_importances
        # Mean of feature importance over trials
        df_feature_importance_mean = df_feature_importance_tree.drop('column_name', axis=1)
        df_feature_importance_mean = df_feature_importance_mean.T
        df_feature_importance_mean.columns = df_feature_importance_tree['column_name']
        df_feature_importance_mean = df_feature_importance_mean.astype('float')
        df_feature_importance_mean_describe = df_feature_importance_mean.describe().T
        df_feature_importance_mean_describe.sort_values('mean', ascending=False, inplace=True)
        # print(df_feature_importance_mean_describe.columns)
        df_feature_importance_mean_describe = df_feature_importance_mean_describe[['count', 'mean']]
        # print_df(df_feature_importance_mean_describe)
        df_feature_importance_mean_describe = df_feature_importance_mean_describe.merge(missing_df, on='column_name')
        return df_feature_importance_mean_describe
        

class ExperimentRunner:
    def __init__(self, 
                 classifier, 
                 experiment_data_helper: ExperimentDataHelper, 
                 num_folds: int = 10, 
                 test_n_folds: int = 1, 
                 n_repeats: int = 1, 
                 display: bool = False) -> None:
        self.classifier = clone(classifier) 
        self.experiment_data_helper = experiment_data_helper
        self.display = display
        # Instead of running all folds just run several folds so that it is easier to bench test
        self.num_folds = num_folds
        self.test_n_folds = test_n_folds
        if self.test_n_folds is None:
            self.test_n_folds = self.num_folds * self.n_repeats
        # TODO: RepeatedStratifiedKFold instead of StratifiedKFold to run experiments
        self.n_repeats = n_repeats
        self.cv_analytics_util = None

    # TODO: make these ones polymorphic
    def get_analytics_util(self) -> AnalyticsUtil:
        return AnalyticsUtil

    def get_cv_analytics_util(self) -> CvAnalyticsUtil:
        return CvAnalyticsUtil

    def run_experiment(self) -> ExperimentRunner:
        data_util_lambdas = experiment_data_helper.train_test_split_util.split_kfold(self.num_folds)
        analytics_utils = []
        for i in range(self.test_n_folds):
            data_util = data_util_lambdas[0]
            analytics_util = self.get_analytics_util()(self.classifier, data_util).fit()
            analytics_utils.append(analytics_util)
        self.cv_analytics_util = self.get_cv_analytics_util()(analytics_utils)
        return self
        

In [ ]:
def scale_features(df):
    sc = StandardScaler()
    df_scaled = df.copy()
    df_scaled = sc.fit_transform(df_scaled)
    df_scaled = pd.DataFrame(df_scaled, columns=df.columns, index=df.index)
    return df_scaled

def get_nearest_neighbors(df1, df2, top=5):
    df1 = df1.drop_duplicates()
    df2 = df2.drop_duplicates()
    df1 = scale_features(df1)
    df2 = scale_features(df2)
    euclidean_distances = []
    indexes = []
    for i in range(len(df1)):
        row1 = df1.iloc[i]
        distances = []
        for j, row2 in df2.iterrows():
            distances.append((j, distance.euclidean(row1, row2)))
        distances = sorted(distances, key=lambda x: x[1], reverse=False)[:top]
        distances = pd.DataFrame(distances, columns=['index', 'distance'])
        indexes.append((distances['index'].to_list()))
        euclidean_distances.append(distances['distance'].to_list())
    return euclidean_distances, indexes

def get_high_confidence_errors(classifier, X_train, X_test, y_train, y_test, label, label_val=0):
    # Insert predicted class and its likelihood
    X_train = X_train.copy()
    X_test = X_test.copy()
    y_pred = classifier.predict(X_test)
    y_prob = classifier.predict_proba(X_test)[:,1]
    X_test_mismatch = X_test.copy()
    X_test_mismatch[label] = y_test
    X_test_mismatch[f'{label}_pred'] = y_pred
    X_test_mismatch[f'{label}_prob'] = y_prob
    X_test_mismatch = X_test_mismatch.drop_duplicates()
    X_test_mismatch = X_test_mismatch[X_test_mismatch[label] != X_test_mismatch[f'{label}_pred']]
    
    # X_test_high_conf = X_test_mismatch[(X_test_mismatch[f'{label}_prob'] < 0.2) | (X_test_mismatch[f'{label}_prob'] > 0.8)]
    X_test_high_conf = X_test_mismatch
    X_test_high_conf = X_test_high_conf[X_test_high_conf[f'{label}_pred'] == label_val]
    
    # Select 5 nearest neightbors 
    X_train[label] = y_train
    X_train_filtered = X_train[X_train[label] == label_val].drop(label, axis=1)
    # X_test_high_conf = X_test.loc[X_test_high_conf.index, :]
    # Calculated euclidean distances
    distances, indices = get_nearest_neighbors(X_test.loc[X_test_high_conf.index, :], X_train_filtered)
    fp_mismatches = []
    X_train[f'{label}_pred'] = -1
    X_train[f'{label}_prob'] = -1
    X_train = X_train.drop_duplicates()
    # print_df(X_train)
    for i in range(len(X_test_high_conf)):
        idx = indices[i]
        missed_record = X_test_high_conf.iloc[[i], :]
        missed_record['distance'] = 0
        close_records = X_train.loc[idx, :]
        close_records['distance'] = distances[i]
        fp_mismatches.append((missed_record, close_records))
    return fp_mismatches

def run_classifiers(classifiers, df, label, strategy='median', drop_n_features = 1, num_folds=10, cv=False, plot_tree=False, differentiate_confusion_matrix_over=None):
    df_feature_importance_tree = None
    df_top_feature_importances = []
    for i in range(drop_n_features):
        
        train_test_lambda, k_fold_lambdas = process_and_impute_for_label_kfold(df, label, strategy, n_max_per_class=10000, num_folds=num_folds, differentiate_confusion_matrix_over=differentiate_confusion_matrix_over)
        X_train, X_test, y_train, y_test, differentiated_test_sets = train_test_lambda()
        auc_dict = {}
        accuracy_dict = {}
        top_feature_importance = []
        for classifier_type, classifier in classifiers.items():
            # Cross Validation
            if cv:
                cv_scores, measures_df, df_feature_importance_mean_describe = get_cv_scores(classifier, k_fold_lambdas)
                cv_scores.to_csv(f'./cv_scores/cv_scores_for_{classifier_type}_{label}.csv')
                measures_df.to_csv(f'./cv_scores/cv_stats_for_{classifier_type}_{label}.csv')
                if classifier_type in ['DecisionTreeClassifier', 'XGBClassifier']:
                    df_feature_importance_mean_describe.to_csv(f'./feature_importance/feature_importance_mean_{classifier_type}_{label}__{num_folds}_trials.csv')

            # Test classifier
            print(classifier_type)
            trained_classifier, auc, accuracy, threshold, report = run_classifier(classifier, X_train, X_test, y_train, y_test)
            print(threshold)
            filtered_on = list(itertools.chain.from_iterable([zip([key]*len(vals), vals) for key, vals in differentiate_confusion_matrix_over.items()]))
            for i, (X_test_filtered, y_test_filtered) in enumerate(differentiated_test_sets):
                print(f'Filtered on: {filtered_on[i]}')                
                y_pred = trained_classifier.predict(X_test_filtered)
                y_prob = trained_classifier.predict_proba(X_test_filtered)[:,1]
                performance_analysis(y_pred, y_prob, y_test_filtered, show_graph=True)

            # Decision Tree feature selection
            if classifier_type in ['DecisionTreeClassifier', 'XGBClassifier']:
                fn=X_train.columns
                top_feature_stats, feature_importances = feature_selection(classifier, fn, accuracy, plot_tree=plot_tree,
                                                                                         filepath=f'./trees/decision_tree_for_{classifier_type}_{label}.png')
                df_top_feature_importances.append(top_feature_stats)
                top_feature_importance.append(feature_importances.iloc[0]['column_name'])
                feature_importances = feature_importances[feature_importances['importance'] > 0]
                if df_feature_importance_tree is not None:
                    df_feature_importance_tree = df_feature_importance_tree.merge(feature_importances, on='column_name', how='outer', suffixes=[f'_tiral_{i}', f'_tiral_{i+1}'])
                else:
                    df_feature_importance_tree = feature_importances
            auc_dict[classifier_type] = auc
            accuracy_dict[classifier_type] = accuracy
            
            fp_mismatches = get_high_confidence_errors(classifier, X_train, X_test, y_train, y_test, label, label_val=0)
        # df = df.drop(top_feature_importance, axis=1)

    return df_top_feature_importances, df_feature_importance_tree, fp_mismatches

